### Naive Bayes classifier (0.79558)

In [ ]:
#initialise tf-idf vectoriser to enable fitting of training text data
tfidf_vectoriser = TfidfVectorizer()

#fit and transform the text data for the training set
tfidf_xtrain = tfidf_vectoriser.fit_transform(X_train)
tfidf_xtest = tfidf_vectoriser.transform(X_test)
tfidf_test = tfidf_vectoriser.transform(df_test['text'].values)
tfidf_xtrain.shape

In [ ]:
tfidf_xtest.shape

(1523, 14492)

In [ ]:
tfidf_test

<3263x14492 sparse matrix of type '<class 'numpy.float64'>'
	with 25537 stored elements in Compressed Sparse Row format>

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_train['text'].values, df_train['target'].values,test_size=0.2, random_state=156)

X_train

array(['corleonedaboss isnt anything desolate bc nature significance first country',
       'fire hazard associated installation noncompliant external cladding highrise buildings insurance aust URL',
       'libertygeek something kissing ass mass murderer sit right potus feels ok URL',
       ...,
       'young children among rescued capsized boat libya URL via irishtimesworld',
       'scriptettesar katiecool btw th actually next year casualty began th september marks years',
       'thetimepast saalon childhood trauma resolved actual trauma fricken babies'],
      dtype=object)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline

naive_bayes =MultinomialNB()
naive_bayes.fit(tfidf_xtrain, y_train)
preds = naive_bayes.predict(tfidf_xtest)

print("naive bayes model accuracy: ", accuracy_score(y_test, preds))

naive bayes model accuracy:  0.804333552199606


In [ ]:
test_preds = naive_bayes.predict(tfidf_test)

In [ ]:
test_preds[:5]

array([1, 0, 1, 1, 1])

In [ ]:
sub = pd.read_csv('./sample_submission (3).csv')
sub.head()

,id,target
0,0,0
1,2,0
2,3,0
3,9,0
4,11,0


In [ ]:
sub['target']= test_preds
sub.head() # public score: 0.79558

,id,target
0,0,1
1,2,0
2,3,1
3,9,1
4,11,1


In [ ]:
sub.to_csv('naive_bayes.csv', index=False)

### GRU

In [ ]:
from keras.layers import GRU

# 모델 생성
model = Sequential()
model.add(Embedding(input_dim=39944 + 1, output_dim=64))
model.add(GRU(64))  # LSTM 대신 GRU를 사용
model.add(Dense(1, activation='sigmoid'))

# 모델 컴파일
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 모델 훈련
model.fit(xtrain_pad, y_train, epochs=10, batch_size=2)

# 모델 예측 및 평가
preds = model.predict(xtest_pad)
preds = (preds > 0.5).astype(int)  # 이진 클래스 레이블로 변환

# 정확도 계산
print("GRU accuracy: ", accuracy_score(y_test, preds))

Epoch 1/10
3045/3045 [==============================] - 151s 49ms/step - loss: 0.6855 - accuracy: 0.5668
Epoch 2/10
3045/3045 [==============================] - 148s 49ms/step - loss: 0.5545 - accuracy: 0.7269
Epoch 3/10
3045/3045 [==============================] - 148s 49ms/step - loss: 0.2946 - accuracy: 0.8790
Epoch 4/10
3045/3045 [==============================] - 150s 49ms/step - loss: 0.1831 - accuracy: 0.9255
Epoch 5/10
3045/3045 [==============================] - 152s 50ms/step - loss: 0.1371 - accuracy: 0.9368
Epoch 6/10
3045/3045 [==============================] - 149s 49ms/step - loss: 0.1127 - accuracy: 0.9414
Epoch 7/10
3045/3045 [==============================] - 150s 49ms/step - loss: 0.1001 - accuracy: 0.9450
Epoch 8/10
3045/3045 [==============================] - 149s 49ms/step - loss: 0.0902 - accuracy: 0.9489
Epoch 9/10
3045/3045 [==============================] - 203s 67ms/step - loss: 0.0875 - accuracy: 0.9506
Epoch 10/10
48/48 [==============================] - 1s

In [ ]:
# Define Embedding layer as pre-processing layer for tokenization
max_features = 20000 #  20000 most frequent words in the input text data.
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
vectorizer = TextVectorization(max_tokens=max_features, output_sequence_length=200, output_mode='int')

X_train = df_train["text"].values
X_test = df_test["text"].values
vectorizer.adapt(np.hstack((X_train, X_test)))
vectorizerd_text = vectorizer(X_train)
vectorizerd_test_text = vectorizer(X_test)
preds = []

for input_text in vectorizerd_test_text:
    pred = model.predict(np.expand_dims(input_text, 0))
    preds.append(pred)

preds = np.round(np.array(preds))
sub['target'] = preds.flatten()
su['target'] = sub['target'].astype('int')
sub.to_csv('GRU.csv', index=False)

1/1 [==============================] - 0s 51ms/step


NameError: name 'sub_sample' is not defined

In [ ]:
preds.shape

(3263, 1, 1)

In [ ]:
sub.head()

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1


In [ ]:
sub['target'] = preds.flatten()
sub['target'] = sub['target'].astype('int')
sub.to_csv('GRU.csv', index=False)

In [ ]:
sub['target'].mean()

1.0

### SimpleRNN (캐글 제출X)

In [ ]:
texts = df_train['text']+df_test['text']

In [ ]:
len(words_list)

39944

In [ ]:
tokenizer = Tokenizer()
tokenizer(texts)

# 텍스트 데이터를 정수 시퀀스로 변환
x_train_seq = tokenizer.texts_to_sequences(X_train)
x_test_seq = tokenizer.texts_to_sequences(X_test)
test_seq = tokenizer.texts_to_sequences(df_test['text'])

# 시퀀스 데이터 패딩
maxlen = 30
x_train_pad = pad_sequences(x_train_seq, maxlen=maxlen)
x_test_pad = pad_sequences(x_test_seq, maxlen=maxlen)
test_pad = pad_sequences(test_seq, maxlen=maxlen)

TypeError: 'Tokenizer' object is not callable

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing import text, sequence
xtrain, xtest, ytrain, ytest = train_test_split(df_train['text'].values, df_train['target'].values, shuffle=True, test_size=0.2)

max_len = max(df_train['text'].apply(lambda x: len(x.split())).values)
max_words = 20000
tokenizer = text.Tokenizer(num_words = max_words)
# create the vocabulary by fitting on x_train text
tokenizer.fit_on_texts(xtrain)
# generate the sequence of tokens
xtrain_seq = tokenizer.texts_to_sequences(xtrain)
xtest_seq = tokenizer.texts_to_sequences(xtest)

# pad the sequences
xtrain_pad = sequence.pad_sequences(xtrain_seq, maxlen=max_len)
xtest_pad = sequence.pad_sequences(xtest_seq, maxlen=max_len)
word_index = tokenizer.word_index

print('text example:', xtrain[0])
print('sequence of indices(before padding):', xtrain_seq[0])
print('sequence of indices(after padding):', xtrain_pad[0])

text example: crossborder terrorism pakistan caught redhanded URL
sequence of indices(before padding): [3608, 264, 749, 597, 3609, 1]
sequence of indices(after padding): [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0 3608  264  749  597 3609    1]


In [ ]:
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN, Dense

model = Sequential()
model.add(Embedding(input_dim = 39944+1, output_dim=64))
model.add(SimpleRNN(64))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


model.fit(xtrain_pad, y_train)
preds = model.predict(xtest_pad)
preds = (preds > 0.5).astype(int)
print("RNN accuracy: ", accuracy_score(y_test, preds)) # 예측 결과가 너무 낮아 캐글 결과 얻지 않음

48/48 [==============================] - 0s 5ms/step
RNN accuracy:  0.5692711753118844
